In [12]:
from pyspark.sql import SparkSession
import os
from pyspark.sql.functions import col

# --- Cấu hình giữ nguyên ---
ORACLE_USER = os.getenv("ORACLE_USER", "DCC_TAISUN")
ORACLE_PWD = os.getenv("ORACLE_PASSWORD", "dcc_taisun")
ORACLE_URL = "jdbc:oracle:thin:@10.0.0.250:1521/TOPPROD"
DRIVER_CLASS = "oracle.jdbc.driver.OracleDriver"
ojdbc_path = "/opt/spark/jars/ojdbc8.jar"


def get_spark_session():
    return SparkSession.builder \
        .appName("Oracle_Inventory_Check") \
        .master("spark://spark-master:7077") \
        .config("spark.hadoop.fs.s3a.access.key", os.environ.get("MINIO_ACCESS_KEY", "minio_access_key")) \
        .config("spark.hadoop.fs.s3a.secret.key", os.environ.get("MINIO_SECRET_KEY", "minio_secret_key")) \
        .config("spark.hadoop.fs.s3a.endpoint", os.environ.get("S3_ENDPOINT", "http://minio:9000")) \
        .config("spark.hadoop.fs.s3a.path.style.access", "true") \
        .config("spark.hadoop.fs.s3a.connection.ssl.enabled", "false") \
        .config("spark.hadoop.fs.s3a.aws.credentials.provider", "org.apache.hadoop.fs.s3a.SimpleAWSCredentialsProvider") \
        .config("spark.hadoop.fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem")\
        .config("spark.hadoop.fs.s3a.fast.upload", "true")\
        .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
        .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog")\
        .config("spark.delta.logStore.class", "org.apache.spark.sql.delta.storage.S3SingleDriverLogStore")\
        .config("spark.driver.extraClassPath", ojdbc_path) \
        .config("spark.executor.extraClassPath", ojdbc_path) \
        .config("spark.jars", ojdbc_path) \
        .getOrCreate()

In [13]:
def get_all_tables_metadata(spark):
    """
    Lấy danh sách bảng, dung lượng, số dòng (ước tính) và số cột
    bằng cách query System View của Oracle.
    """
    
    # SQL Query "Thần thánh" để lấy Metadata
    # - ALL_TABLES: Lấy tên bảng và số dòng (Last Analyzed)
    # - ALL_TAB_COLUMNS: Đếm số cột
    # - USER_SEGMENTS: Tính dung lượng đĩa cứng (Bytes)
    inventory_query = f"""
    (
        SELECT 
            t.TABLE_NAME,
            t.NUM_ROWS AS APPROX_ROW_COUNT,
            t.LAST_ANALYZED,
            NVL(c.COL_COUNT, 0) AS COL_COUNT,
            ROUND(NVL(s.BYTES, 0) / 1024 / 1024, 2) AS SIZE_MB
        FROM ALL_TABLES t
        -- Join để đếm số cột
        LEFT JOIN (
            SELECT TABLE_NAME, COUNT(*) AS COL_COUNT 
            FROM ALL_TAB_COLUMNS 
            WHERE OWNER = '{ORACLE_USER}' 
            GROUP BY TABLE_NAME
        ) c ON t.TABLE_NAME = c.TABLE_NAME
        -- Join để tính dung lượng
        LEFT JOIN (
            SELECT SEGMENT_NAME, SUM(BYTES) AS BYTES 
            FROM USER_SEGMENTS 
            GROUP BY SEGMENT_NAME
        ) s ON t.TABLE_NAME = s.SEGMENT_NAME
        WHERE t.OWNER = '{ORACLE_USER}'
        ORDER BY SIZE_MB DESC
    ) tmp
    """

    print(">>> Đang lấy Metadata từ Oracle System Views...")
    
    df = spark.read \
        .format("jdbc") \
        .option("url", ORACLE_URL) \
        .option("dbtable", inventory_query) \
        .option("user", ORACLE_USER) \
        .option("password", ORACLE_PWD) \
        .option("driver", DRIVER_CLASS) \
        .load()
        
    return df

In [14]:
spark = get_spark_session()

In [15]:
df_inventory = get_all_tables_metadata(spark)

>>> Đang lấy Metadata từ Oracle System Views...


In [16]:
print("\n>>> TOP 20 BẢNG CÓ DUNG LƯỢNG LỚN NHẤT:")
df_inventory.select("TABLE_NAME", "SIZE_MB", "APPROX_ROW_COUNT", "COL_COUNT").show(20, False)


>>> TOP 20 BẢNG CÓ DUNG LƯỢNG LỚN NHẤT:
+-----------------------+---------------+------------------+--------------+
|TABLE_NAME             |SIZE_MB        |APPROX_ROW_COUNT  |COL_COUNT     |
+-----------------------+---------------+------------------+--------------+
|TLF_FILE               |1092.0000000000|2634090.0000000000|111.0000000000|
|CCH_FILE               |768.0000000000 |1996382.0000000000|129.0000000000|
|TLFC_FILE              |616.0000000000 |2355819.0000000000|52.0000000000 |
|TC_BAK_INVOICE_LIST    |480.0000000000 |514205.0000000000 |122.0000000000|
|OGB_FILE               |360.0000000000 |1123467.0000000000|104.0000000000|
|SFE_FILE               |208.0000000000 |989824.0000000000 |62.0000000000 |
|OEB_FILE               |192.0000000000 |564194.0000000000 |121.0000000000|
|NPQ_FILE               |184.0000000000 |725910.0000000000 |37.0000000000 |
|TT112633744_AXMQ001_TMP|181.0000000000 |0.0000000000      |63.0000000000 |
|ABB_FILE               |176.0000000000 |838682

In [18]:
# In ra các cấu hình quan trọng liên quan đến S3/MinIO
print("--- CẤU HÌNH S3A HIỆN TẠI ---")

keys_to_check = [
    "spark.hadoop.fs.s3a.endpoint",
    "spark.hadoop.fs.s3a.access.key",
    "spark.hadoop.fs.s3a.secret.key",
    "spark.hadoop.fs.s3a.path.style.access",
    "spark.hadoop.fs.s3a.connection.ssl.enabled",
    "spark.hadoop.fs.s3a.impl",
    "spark.hadoop.fs.s3a.aws.credentials.provider"
]

for key in keys_to_check:
    # Dùng tham số thứ 2 là giá trị mặc định nếu không tìm thấy key
    val = spark.conf.get(key, "KHÔNG TÌM THẤY")
    print(f"{key}: {val}")

--- CẤU HÌNH S3A HIỆN TẠI ---
spark.hadoop.fs.s3a.endpoint: http://minio:9000
spark.hadoop.fs.s3a.access.key: minio_access_key
spark.hadoop.fs.s3a.secret.key: minio_secret_key
spark.hadoop.fs.s3a.path.style.access: true
spark.hadoop.fs.s3a.connection.ssl.enabled: false
spark.hadoop.fs.s3a.impl: org.apache.hadoop.fs.s3a.S3AFileSystem
spark.hadoop.fs.s3a.aws.credentials.provider: org.apache.hadoop.fs.s3a.SimpleAWSCredentialsProvider


In [19]:
print("--- BIẾN MÔI TRƯỜNG (OS ENV) ---")
env_vars = ["AWS_ACCESS_KEY_ID", "AWS_SECRET_ACCESS_KEY", "MINIO_ACCESS_KEY", "MINIO_SECRET_KEY"]

for var in env_vars:
    print(f"{var}: {os.environ.get(var, 'Không thiết lập')}")

--- BIẾN MÔI TRƯỜNG (OS ENV) ---
AWS_ACCESS_KEY_ID:  minio_access_key
AWS_SECRET_ACCESS_KEY:  minio_secret_key
MINIO_ACCESS_KEY: minio_access_key
MINIO_SECRET_KEY: minio_secret_key


In [17]:
df_inventory.write.format("delta").mode("overwrite").save("s3a://datalake/bronze/metadata/oracle_tables")

Py4JJavaError: An error occurred while calling o157.save.
: java.nio.file.AccessDeniedException: s3a://datalake/bronze/metadata/oracle_tables/_delta_log: getFileStatus on s3a://datalake/bronze/metadata/oracle_tables/_delta_log: com.amazonaws.services.s3.model.AmazonS3Exception: Forbidden (Service: Amazon S3; Status Code: 403; Error Code: 403 Forbidden; Request ID: 8ETPRE2ZPZYSBQYW; S3 Extended Request ID: S0BEervNS8siMTr+ZSaxeunwoBB+GoRr3QqCyW+SHBn/ZAtcuN6C5JcWVwjpt9nN8IxGsi6vNfYtdFxUgTlLow==; Proxy: null), S3 Extended Request ID: S0BEervNS8siMTr+ZSaxeunwoBB+GoRr3QqCyW+SHBn/ZAtcuN6C5JcWVwjpt9nN8IxGsi6vNfYtdFxUgTlLow==:403 Forbidden
	at org.apache.hadoop.fs.s3a.S3AUtils.translateException(S3AUtils.java:255)
	at org.apache.hadoop.fs.s3a.S3AUtils.translateException(S3AUtils.java:175)
	at org.apache.hadoop.fs.s3a.S3AFileSystem.s3GetFileStatus(S3AFileSystem.java:3796)
	at org.apache.hadoop.fs.s3a.S3AFileSystem.innerGetFileStatus(S3AFileSystem.java:3688)
	at org.apache.hadoop.fs.s3a.S3AFileS

In [ ]:
    df_inventory = get_all_tables_metadata(spark)
    
    # 1. Hiển thị Top 20 bảng nặng nhất
    print("\n>>> TOP 20 BẢNG CÓ DUNG LƯỢNG LỚN NHẤT:")
    df_inventory.select("TABLE_NAME", "SIZE_MB", "APPROX_ROW_COUNT", "COL_COUNT").show(20, False)
    
    # 2. (Optional) Lưu danh sách này xuống Bronze để làm tài liệu (Data Catalog)
    # df_inventory.write.format("delta").mode("overwrite").save("s3a://datalake/bronze/metadata/oracle_tables")
    
   

In [9]:
df_empty_tables = df_inventory.filter(
        (col("APPROX_ROW_COUNT") == 0) | 
        (col("APPROX_ROW_COUNT").isNull())
    )
    
# 3. Sắp xếp theo tên bảng cho dễ nhìn
df_empty_tables = df_empty_tables.orderBy("TABLE_NAME")

print(f"\n>>> TÌM THẤY {df_empty_tables.count()} BẢNG KHÔNG CÓ DỮ LIỆU (HOẶC CHƯA ANALYZE):")
df_empty_tables.select("TABLE_NAME", "APPROX_ROW_COUNT", "LAST_ANALYZED").show(100, False)


>>> TÌM THẤY 5243 BẢNG KHÔNG CÓ DỮ LIỆU (HOẶC CHƯA ANALYZE):
+---------------+----------------+-------------------+
|TABLE_NAME     |APPROX_ROW_COUNT|LAST_ANALYZED      |
+---------------+----------------+-------------------+
|250701_DU2     |0.0000000000    |2025-07-01 22:00:25|
|AAB_FILE       |0.0000000000    |2019-12-19 22:00:13|
|AAI_FILE       |0.0000000000    |2019-12-19 22:00:13|
|AAJ_FILE       |0.0000000000    |2019-12-19 22:00:13|
|AAK_FILE       |0.0000000000    |2019-12-19 22:00:13|
|AAL_FILE       |0.0000000000    |2019-12-19 22:00:13|
|AAW_FILE       |0.0000000000    |2019-12-19 22:00:13|
|AAX_FILE       |0.0000000000    |2019-12-19 22:00:13|
|AAY_FILE       |0.0000000000    |2019-12-19 22:00:13|
|ABA_TK_FILE    |0.0000000000    |2020-08-27 22:00:33|
|ABB_PB_FILE    |0.0000000000    |2023-03-23 22:00:26|
|ABC_FILE       |0.0000000000    |2019-12-19 22:00:13|
|ABG_FILE       |0.0000000000    |2019-12-19 22:00:13|
|ABH_FILE       |0.0000000000    |2019-12-19 22:00:13|
|AB

In [11]:
spark.stop()